# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

Prepare the training and testing data for the model.
This model will use a rolling 10 day window to predict the 11th day closing price.

1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model.

Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [33]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [34]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [35]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [36]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [37]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [38]:
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [39]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [40]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [41]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [42]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)


In [43]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

Design a custom LSTM RNN and fit (train) it using the training data.

1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
We want to use the same model architecture and random seed for both notebooks.
This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [45]:
# Build the LSTM model. 

model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
    return_sequences=True,
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features
    input_shape=(X_train.shape[1], 1))
    )
# Note: The dropouts help prevent overfitting
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))


In [46]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [47]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 5)             140       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 5)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 5)             220       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 5)             0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dropout_5 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [48]:
# Train the model
model.fit(
    X_train,
    y_train,
# Use at least 10 epochs
    epochs=10,
# Do not shuffle the data
    shuffle=False,
# Experiement with the batch size, but a smaller batch size is recommended
    batch_size=1,
    verbose=1)

Epoch 1/10
372/372 [==============================] - 1s 4ms/step - loss: 0.0617
Epoch 2/10
372/372 [==============================] - 1s 4ms/step - loss: 0.0317
Epoch 3/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0285
Epoch 4/10
372/372 [==============================] - 1s 4ms/step - loss: 0.0232
Epoch 5/10
372/372 [==============================] - 1s 4ms/step - loss: 0.0192
Epoch 6/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0163
Epoch 7/10
372/372 [==============================] - 1s 4ms/step - loss: 0.0162
Epoch 8/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0139
Epoch 9/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0121
Epoch 10/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0115


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [49]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 2ms/step - loss: 0.0444


0.044422559440135956

In [50]:
# Make some predictions
predicted = model.predict(X_test)

In [51]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [53]:
# Create a DataFrame of Real and Predicted values
crypto = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
crypto.tail()

,Real,Predicted
2019-07-25,9772.139648,7446.352539
2019-07-26,9882.429688,7442.677734
2019-07-27,9847.450195,7422.921387
2019-07-28,9478.320313,7416.961426
2019-07-29,9531.769531,7391.745117


In [54]:
# Plot the real vs predicted values as a line chart
crypto.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [75]:
# Calculate percentage tof Real value that Predicted value holds and add it to a new column
crypto["Percent of Real"] = (crypto["Predicted"] / crypto["Real"])

In [76]:
# Calculate average percentage of Predicted value to Real value (starting after data values diverge)
crypto.loc['2019-05-01':,["Percent of Real"]].mean()

Percent of Real    0.738546
dtype: float64